# Conventional Water Treatment Plant using WaterTAP

This notebook demonstrates a complete conventional water treatment plant model including:
- Coagulation/Flocculation
- Sedimentation (Clarifier)
- Filtration
- Disinfection

## Installation
Make sure WaterTAP is installed:
```bash
pip install watertap
pip install ipopt
conda install -c conda-forge ipopt -y
conda install -c conda-forge cyipopt -y
```

## Import Required Libraries

In [3]:
# Core Pyomo and IDAES imports
from pyomo.environ import (
    ConcreteModel,
    TransformationFactory,
    units as pyunits,
    value,
    SolverFactory,
)
from pyomo.network import Arc
from idaes.core import FlowsheetBlock
from idaes.core.solvers import get_solver
from idaes.core.util.initialization import propagate_state
from idaes.core.util.model_statistics import degrees_of_freedom

# IDAES unit models
from idaes.models.unit_models import Mixer, Separator, CSTR, Feed, Product
from idaes.models.unit_models.mixer import MomentumMixingType

# WaterTAP imports
from watertap.core.util.initialization import check_solve
from watertap.unit_models.clarifier import Clarifier
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock

# Check available solvers
print("Checking available solvers:")
for solver_name in ['ipopt', 'glpk', 'cbc', 'gurobi', 'cplex']:
    solver = SolverFactory(solver_name)
    print(f"  {solver_name}: {solver.available()}")

Checking available solvers:
  ipopt: True
  glpk: False
  cbc: False
  gurobi: False
  cplex: False


## Define Function to Create Water Treatment Plant

In [25]:
import sys
from pathlib import Path

# current working dir in this notebook = WaterTAP-codes/notebooks
notebooks_dir = Path.cwd()
project_root = notebooks_dir.parent            # WaterTAP-codes
src_path = project_root / "src"

# Make sure src/ is on the Python path
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from wtp_models.baseline_model import example_flow_balance



ModuleNotFoundError: No module named 'wtp_models'

In [ ]:
def create_water_treatment_plant():
    """
    Create a conventional water treatment plant flowsheet
    """
    # Create model and flowsheet
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Property package for water treatment
    m.fs.properties = NaClParameterBlock()
    
    # Unit operations
    print("Building unit operations...")
    
    # 1. Raw water feed
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # 2. Coagulation/Flocculation mixer (chemical addition)
    m.fs.coagulation = Mixer(
        property_package=m.fs.properties,
        inlet_list=["water", "coagulant"],
        momentum_mixing_type=MomentumMixingType.equality,
    )
    
    # 3. Flocculation tank (modeled as simple mixer for residence time)
    m.fs.flocculation_tank = Mixer(
        property_package=m.fs.properties,
        inlet_list=["inlet"],
        momentum_mixing_type=MomentumMixingType.equality,
    )
    
    # 4. Sedimentation/Clarifier (using Separator as a simplified model)
    m.fs.clarifier = Separator(
        property_package=m.fs.properties,
        outlet_list=["effluent", "underflow"],
    )
    
    # 5. Filtration (modeled as separator for simplicity)
    m.fs.filtration = Separator(
        property_package=m.fs.properties,
        outlet_list=["treated_water", "backwash"],
    )
    
    # 6. Disinfection tank (modeled as mixer)
    m.fs.disinfection = Mixer(
        property_package=m.fs.properties,
        inlet_list=["inlet"],
        momentum_mixing_type=MomentumMixingType.equality,
    )
    
    # 7. Treated water product
    m.fs.product = Product(property_package=m.fs.properties)
    
    # 8. Sludge output
    m.fs.sludge = Product(property_package=m.fs.properties)
    
    # Connect units with arcs
    print("Connecting units...")
    
    m.fs.feed_to_coag = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.coagulation.water,
    )
    
    m.fs.coag_to_floc = Arc(
        source=m.fs.coagulation.outlet,
        destination=m.fs.flocculation_tank.inlet,
    )
    
    m.fs.floc_to_clarifier = Arc(
        source=m.fs.flocculation_tank.outlet,
        destination=m.fs.clarifier.inlet,
    )
    
    # Clarifier has 'effluent' (treated) and 'underflow' (sludge) outlets
    m.fs.clarifier_to_filter = Arc(
        source=m.fs.clarifier.effluent,
        destination=m.fs.filtration.inlet,
    )
    
    m.fs.filter_to_disinfect = Arc(
        source=m.fs.filtration.treated_water,
        destination=m.fs.disinfection.inlet,
    )
    
    m.fs.disinfect_to_product = Arc(
        source=m.fs.disinfection.outlet,
        destination=m.fs.product.inlet,
    )
    
    m.fs.clarifier_to_sludge = Arc(
        source=m.fs.clarifier.underflow,
        destination=m.fs.sludge.inlet,
    )
    
    # Expand arcs
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    print("Model created successfully!")
    return m

## Set Operating Conditions

In [ ]:
def set_operating_conditions(m):
    """
    Set operating conditions for the treatment plant - CORRECTED VERSION
    """
    print("Setting operating conditions...")
    
    # Feed water conditions (raw water)
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(100)  # kg/s
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(3.5)  # kg/s (TDS)
    m.fs.feed.properties[0].temperature.fix(298.15)  # K
    m.fs.feed.properties[0].pressure.fix(101325)  # Pa
    
    # Coagulant addition (simplified)
    m.fs.coagulation.coagulant.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.5)  # kg/s
    m.fs.coagulation.coagulant.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(0.05)  # kg/s
    m.fs.coagulation.coagulant.temperature[0].fix(298.15)
    m.fs.coagulation.coagulant.pressure[0].fix(101325)
    
    # CRITICAL: Only fix ONE split fraction per separator
    # The other is automatically 1 - fixed_value
    m.fs.clarifier.split_fraction[0, "effluent"].fix(0.90)
    # DO NOT FIX: m.fs.clarifier.split_fraction[0, "underflow"]
    
    m.fs.filtration.split_fraction[0, "treated_water"].fix(0.99)
    # DO NOT FIX: m.fs.filtration.split_fraction[0, "backwash"]
    
    print("✓ Operating conditions set (only 1 split fraction per unit)")

## Initialize the Model

In [ ]:
def initialize_model(m):
    """
    Initialize the flowsheet
    """
    print("Initializing model...")
    
    # Initialize feed
    m.fs.feed.initialize()
    propagate_state(m.fs.feed_to_coag)
    
    # Initialize coagulation with solver name as string
    m.fs.coagulation.initialize(solver='ipopt')
    propagate_state(m.fs.coag_to_floc)
    
    # Initialize flocculation
    m.fs.flocculation_tank.initialize(solver='ipopt')
    propagate_state(m.fs.floc_to_clarifier)
    
    # Initialize clarifier
    m.fs.clarifier.initialize(solver='ipopt')
    propagate_state(m.fs.clarifier_to_filter)
    propagate_state(m.fs.clarifier_to_sludge)
    
    # Initialize filtration
    m.fs.filtration.initialize(solver='ipopt')
    propagate_state(m.fs.filter_to_disinfect)
    
    # Initialize disinfection
    m.fs.disinfection.initialize(solver='ipopt')
    propagate_state(m.fs.disinfect_to_product)
    
    # Initialize products
    m.fs.product.initialize()
    m.fs.sludge.initialize()
    
    print("Initialization complete!")

## Solve the Model

In [ ]:
def solve_model(m):
    """
    Solve the model
    """
    print(f"\nDegrees of freedom: {degrees_of_freedom(m)}")
    
    # Use ipopt solver directly instead of get_solver()
    from pyomo.environ import SolverFactory
    solver = SolverFactory('ipopt')
    
    results = solver.solve(m, tee=True)
    
    from pyomo.opt import TerminationCondition
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolver Status: Optimal solution found")
    else:
        print(f"\nSolver Status: {results.solver.termination_condition}")
    
    return results

## Display Results

In [ ]:
def display_results(m):
    """
    Display key results
    """
    print("\n" + "="*60)
    print("WATER TREATMENT PLANT RESULTS")
    print("="*60)
    
    print("\n--- RAW WATER FEED ---")
    print(f"Flow rate: {value(m.fs.feed.properties[0].flow_vol):.3f} m³/s")
    print(f"TDS concentration: {value(m.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'NaCl']):.1f} mg/L")
    
    print("\n--- AFTER CLARIFICATION ---")
    treated_flow = value(m.fs.clarifier.effluent_state[0].flow_vol)
    print(f"Treated water flow: {treated_flow:.3f} m³/s")
    print(f"Water recovery: {treated_flow/value(m.fs.feed.properties[0].flow_vol)*100:.1f}%")
    
    print("\n--- FINAL PRODUCT WATER ---")
    product_flow = value(m.fs.product.properties[0].flow_vol)
    print(f"Product flow rate: {product_flow:.3f} m³/s")
    print(f"Product TDS: {value(m.fs.product.properties[0].conc_mass_phase_comp['Liq', 'NaCl']):.1f} mg/L")
    print(f"Overall recovery: {product_flow/value(m.fs.feed.properties[0].flow_vol)*100:.1f}%")
    
    print("\n--- SLUDGE OUTPUT ---")
    sludge_flow = value(m.fs.sludge.properties[0].flow_vol)
    print(f"Sludge flow rate: {sludge_flow:.3f} m³/s")
    print(f"Sludge TDS: {value(m.fs.sludge.properties[0].conc_mass_phase_comp['Liq', 'NaCl']):.1f} mg/L")
    
    print("\n" + "="*60)

## Run the Complete Simulation

In [ ]:
print("Creating conventional water treatment plant model...")
try:
    model = create_water_treatment_plant()
    print(f"Model created successfully: {model is not None}")
except Exception as e:
    print(f"Error creating model: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Check what's available on the clarifier
print("Clarifier attributes:")
clarifier_attrs = [attr for attr in dir(model.fs.clarifier) if not attr.startswith('_')]
print(clarifier_attrs[:30])  # Show first 30 attributes

print("\n\nChecking clarifier configuration:")
print(f"Config: {model.fs.clarifier.config}")

print("\n\nChecking if split_fraction exists:")
print(f"Has split_fraction: {hasattr(model.fs.clarifier, 'split_fraction')}")

In [ ]:
# Check clarifier outlet structure
print("Clarifier attributes:")
print([attr for attr in dir(model.fs.clarifier) if 'effluent' in attr.lower() or 'state' in attr.lower()])

print("\nTrying to find flow_vol:")
# Check if there's a mixed_state
if hasattr(model.fs.clarifier, 'mixed_state'):
    print(f"Mixed state flow_vol: {value(model.fs.clarifier.mixed_state[0].flow_vol):.3f}")

# Check outlet states
for attr_name in dir(model.fs.clarifier):
    if 'state' in attr_name.lower() and not attr_name.startswith('_'):
        print(f"\nFound state block: {attr_name}")
        attr = getattr(model.fs.clarifier, attr_name)
        if hasattr(attr, '__getitem__'):
            try:
                state = attr[0]
                if hasattr(state, 'flow_vol'):
                    print(f"  {attr_name}[0].flow_vol = {value(state.flow_vol):.3f}")
            except:
                pass

In [ ]:
print("\nSetting operating conditions...")
set_operating_conditions(model)

In [ ]:
print("\nInitializing flowsheet...")
initialize_model(model)

In [ ]:
print("\nSolving model...")
results = solve_model(model)

In [ ]:
print("\nDisplaying results...")
display_results(model)
print("\n✓ Water treatment plant simulation complete!")

## Optional: Visualize Results

You can add visualization cells here to plot:
- Flow rates through each unit
- TDS concentrations at each stage
- Water recovery rates
- Mass balances

In [ ]:
# Example: Plot flow rates through treatment train
import matplotlib.pyplot as plt

stages = ['Feed', 'After Coag', 'After Clarifier', 'After Filter', 'Product']
flows = [
    value(model.fs.feed.properties[0].flow_vol),
    value(model.fs.coagulation.mixed_state[0].flow_vol),
    value(model.fs.clarifier.effluent_state[0].flow_vol),
    value(model.fs.filtration.treated_water_state[0].flow_vol),
    value(model.fs.product.properties[0].flow_vol)
]

plt.figure(figsize=(10, 6))
plt.plot(stages, flows, marker='o', linewidth=2, markersize=8)
plt.xlabel('Treatment Stage', fontsize=12)
plt.ylabel('Flow Rate (m³/s)', fontsize=12)
plt.title('Water Flow Through Treatment Plant', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
print("=== FLOW BALANCE CHECK ===\n")

print("Feed:")
print(f"  Flow: {value(model.fs.feed.properties[0].flow_vol):.4f} m³/s")

print("\nAfter Coagulation (with coagulant added):")
print(f"  Flow: {value(model.fs.coagulation.mixed_state[0].flow_vol):.4f} m³/s")

print("\nClarifier:")
print(f"  Inlet flow: {value(model.fs.clarifier.mixed_state[0].flow_vol):.4f} m³/s")
print(f"  Effluent flow: {value(model.fs.clarifier.effluent_state[0].flow_vol):.4f} m³/s")
print(f"  Underflow flow: {value(model.fs.clarifier.underflow_state[0].flow_vol):.4f} m³/s")
print(f"  Split fraction (effluent): {value(model.fs.clarifier.split_fraction[0, 'effluent']):.2f}")

print("\nFiltration:")
print(f"  Inlet flow: {value(model.fs.filtration.mixed_state[0].flow_vol):.4f} m³/s")
print(f"  Treated water: {value(model.fs.filtration.treated_water_state[0].flow_vol):.4f} m³/s")
print(f"  Backwash: {value(model.fs.filtration.backwash_state[0].flow_vol):.4f} m³/s")
print(f"  Split fraction (treated): {value(model.fs.filtration.split_fraction[0, 'treated_water']):.2f}")

print("\nProduct:")
print(f"  Flow: {value(model.fs.product.properties[0].flow_vol):.4f} m³/s")

In [ ]:
print(f"Degrees of freedom: {degrees_of_freedom(model)}")
print("\nChecking if model was actually solved...")
print(f"Has results been stored: {'results' in dir()}")

In [ ]:
# 1. Create model
print("Creating model...")
model = create_water_treatment_plant()

# 2. Set operating conditions  
print("\nSetting operating conditions...")
set_operating_conditions(model)

# 3. Check DOF before solving
print(f"\nDegrees of freedom: {degrees_of_freedom(model)}")

# 4. Solve directly without initialization
print("\nSolving model directly...")
from pyomo.environ import SolverFactory
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

print(f"\nSolver status: {results.solver.status}")
print(f"Termination condition: {results.solver.termination_condition}")

In [ ]:
def display_detailed_water_quality(m):
    """
    Display comprehensive water quality parameters at each stage
    """
    print("\n" + "="*80)
    print("DETAILED WATER QUALITY ANALYSIS")
    print("="*80)
    
    # Helper function to get all properties
    def get_properties(state_block):
        props = {}
        props['flow_vol'] = value(state_block.flow_vol)
        props['flow_mass_H2O'] = value(state_block.flow_mass_phase_comp['Liq', 'H2O'])
        props['flow_mass_NaCl'] = value(state_block.flow_mass_phase_comp['Liq', 'NaCl'])
        props['conc_NaCl'] = value(state_block.conc_mass_phase_comp['Liq', 'NaCl'])
        props['temperature'] = value(state_block.temperature)
        props['pressure'] = value(state_block.pressure)
        props['dens_mass'] = value(state_block.dens_mass_phase['Liq'])
        return props
    
    # 1. RAW WATER FEED
    print("\n" + "-"*80)
    print("1. RAW WATER FEED (Upstream of Treatment)")
    print("-"*80)
    feed_props = get_properties(m.fs.feed.properties[0])
    print(f"  Volumetric Flow Rate:     {feed_props['flow_vol']:.4f} m³/s")
    print(f"  Mass Flow - H2O:          {feed_props['flow_mass_H2O']:.3f} kg/s")
    print(f"  Mass Flow - NaCl (TDS):   {feed_props['flow_mass_NaCl']:.3f} kg/s")
    print(f"  TDS Concentration:        {feed_props['conc_NaCl']:.2f} mg/L")
    print(f"  Temperature:              {feed_props['temperature']:.2f} K ({feed_props['temperature']-273.15:.2f} °C)")
    print(f"  Pressure:                 {feed_props['pressure']/1000:.2f} kPa")
    print(f"  Density:                  {feed_props['dens_mass']:.2f} kg/m³")
    
    # 2. AFTER COAGULATION
    print("\n" + "-"*80)
    print("2. AFTER COAGULATION/FLOCCULATION")
    print("-"*80)
    print("  Upstream (Before Coagulant Addition):")
    coag_in_props = feed_props  # Same as feed
    print(f"    Flow Rate:              {coag_in_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {coag_in_props['conc_NaCl']:.2f} mg/L")
    
    print("\n  Downstream (After Coagulant Addition + Mixing):")
    coag_props = get_properties(m.fs.coagulation.mixed_state[0])
    print(f"    Volumetric Flow Rate:   {coag_props['flow_vol']:.4f} m³/s")
    print(f"    Mass Flow - H2O:        {coag_props['flow_mass_H2O']:.3f} kg/s")
    print(f"    Mass Flow - NaCl:       {coag_props['flow_mass_NaCl']:.3f} kg/s")
    print(f"    TDS Concentration:      {coag_props['conc_NaCl']:.2f} mg/L")
    print(f"    Temperature:            {coag_props['temperature']:.2f} K")
    print(f"    Pressure:               {coag_props['pressure']/1000:.2f} kPa")
    print(f"    Density:                {coag_props['dens_mass']:.2f} kg/m³")
    
    # 3. CLARIFIER
    print("\n" + "-"*80)
    print("3. CLARIFIER (Sedimentation)")
    print("-"*80)
    print("  Upstream (Clarifier Inlet):")
    clar_in_props = get_properties(m.fs.clarifier.mixed_state[0])
    print(f"    Flow Rate:              {clar_in_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {clar_in_props['conc_NaCl']:.2f} mg/L")
    
    print("\n  Downstream - Effluent (Clarified Water):")
    clar_eff_props = get_properties(m.fs.clarifier.effluent_state[0])
    print(f"    Flow Rate:              {clar_eff_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {clar_eff_props['conc_NaCl']:.2f} mg/L")
    print(f"    Water Recovery:         {clar_eff_props['flow_vol']/clar_in_props['flow_vol']*100:.1f}%")
    tds_removal_clar = (clar_in_props['conc_NaCl'] - clar_eff_props['conc_NaCl'])/clar_in_props['conc_NaCl']*100
    print(f"    TDS Removal:            {tds_removal_clar:.1f}%")
    
    print("\n  Downstream - Underflow (Sludge):")
    clar_under_props = get_properties(m.fs.clarifier.underflow_state[0])
    print(f"    Flow Rate:              {clar_under_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {clar_under_props['conc_NaCl']:.2f} mg/L")
    print(f"    Sludge Flow:            {clar_under_props['flow_vol']/clar_in_props['flow_vol']*100:.1f}% of inlet")
    
    # 4. FILTRATION
    print("\n" + "-"*80)
    print("4. FILTRATION")
    print("-"*80)
    print("  Upstream (Filter Inlet):")
    filt_in_props = get_properties(m.fs.filtration.mixed_state[0])
    print(f"    Flow Rate:              {filt_in_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {filt_in_props['conc_NaCl']:.2f} mg/L")
    
    print("\n  Downstream - Treated Water:")
    filt_treat_props = get_properties(m.fs.filtration.treated_water_state[0])
    print(f"    Flow Rate:              {filt_treat_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {filt_treat_props['conc_NaCl']:.2f} mg/L")
    print(f"    Water Recovery:         {filt_treat_props['flow_vol']/filt_in_props['flow_vol']*100:.1f}%")
    tds_removal_filt = (filt_in_props['conc_NaCl'] - filt_treat_props['conc_NaCl'])/filt_in_props['conc_NaCl']*100
    print(f"    TDS Removal:            {tds_removal_filt:.1f}%")
    
    print("\n  Downstream - Backwash:")
    filt_back_props = get_properties(m.fs.filtration.backwash_state[0])
    print(f"    Flow Rate:              {filt_back_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {filt_back_props['conc_NaCl']:.2f} mg/L")
    
    # 5. DISINFECTION
    print("\n" + "-"*80)
    print("5. DISINFECTION")
    print("-"*80)
    print("  Upstream (Disinfection Inlet):")
    dis_in_props = filt_treat_props  # Same as filter outlet
    print(f"    Flow Rate:              {dis_in_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {dis_in_props['conc_NaCl']:.2f} mg/L")
    
    print("\n  Downstream (After Disinfection):")
    dis_props = get_properties(m.fs.disinfection.mixed_state[0])
    print(f"    Flow Rate:              {dis_props['flow_vol']:.4f} m³/s")
    print(f"    TDS Concentration:      {dis_props['conc_NaCl']:.2f} mg/L")
    
    # 6. FINAL PRODUCT
    print("\n" + "-"*80)
    print("6. FINAL PRODUCT WATER")
    print("-"*80)
    prod_props = get_properties(m.fs.product.properties[0])
    print(f"  Volumetric Flow Rate:     {prod_props['flow_vol']:.4f} m³/s")
    print(f"  Mass Flow - H2O:          {prod_props['flow_mass_H2O']:.3f} kg/s")
    print(f"  Mass Flow - NaCl:         {prod_props['flow_mass_NaCl']:.3f} kg/s")
    print(f"  TDS Concentration:        {prod_props['conc_NaCl']:.2f} mg/L")
    print(f"  Temperature:              {prod_props['temperature']:.2f} K ({prod_props['temperature']-273.15:.2f} °C)")
    print(f"  Pressure:                 {prod_props['pressure']/1000:.2f} kPa")
    print(f"  Density:                  {prod_props['dens_mass']:.2f} kg/m³")
    
    # OVERALL PERFORMANCE
    print("\n" + "="*80)
    print("OVERALL TREATMENT PERFORMANCE")
    print("="*80)
    overall_recovery = prod_props['flow_vol']/feed_props['flow_vol']*100
    overall_tds_removal = (feed_props['conc_NaCl'] - prod_props['conc_NaCl'])/feed_props['conc_NaCl']*100
    print(f"  Overall Water Recovery:       {overall_recovery:.2f}%")
    print(f"  Overall TDS Removal:          {overall_tds_removal:.2f}%")
    print(f"  Feed TDS:                     {feed_props['conc_NaCl']:.2f} mg/L")
    print(f"  Product TDS:                  {prod_props['conc_NaCl']:.2f} mg/L")
    print(f"  TDS Reduction:                {feed_props['conc_NaCl'] - prod_props['conc_NaCl']:.2f} mg/L")
    print("="*80 + "\n")

In [ ]:
# Run the complete simulation
print("Creating model...")
model = create_water_treatment_plant()

set_operating_conditions(model)
print(f"Degrees of freedom: {degrees_of_freedom(model)}")

if degrees_of_freedom(model) == 0:
    initialize_model(model)
    results = solve_model(model)
    
    # Display detailed water quality
    display_detailed_water_quality(model)
else:
    print(f"ERROR: DOF = {degrees_of_freedom(model)}")